In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from reinforcement.players import DummyPlayer, ModelPlayer
from reinforcement.dqn.dqn_v2 import DQNTrainerV2
from modeling.chessformer import ChessFormerDQN
from torch.optim import AdamW
import torch as t
from copy import deepcopy

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BUFFER_SIZE = 1_000_000
RANDOM_ACTION_RATE = 0.0
BATCH_SIZE = 32
NB_STEPS_REWARD = 1
WARM_UP_STEPS = 20_000
ALPHA_SAMPLING = 0.7
BETA_SAMPLING = 0.5

In [4]:
model_1 = ChessFormerDQN(6, 3, 512, 2048, 64, 8).to('cuda')
model_2 = ChessFormerDQN(6, 3, 512, 2048, 64, 8).to('cuda')
#model_1_state_dict = t.load('runs/experiment_4/model_1.pt', map_location='cuda')
#model_1.load_state_dict(model_state_dict)
optimizer = AdamW(model_1.parameters(), lr=1e-4, eps=1e-1)
competitor = DummyPlayer()

In [5]:
TAU = 0.0001

In [6]:
trainer = DQNTrainerV2(model_1=model_1,
                       model_2=model_2,
                       optimizer=optimizer,
                       buffer_size=BUFFER_SIZE,
                       competitor=competitor,
                       batch_size=BATCH_SIZE,
                       experiment_name='experiment_15',
                       model_device='cuda',
                       nb_steps_reward=NB_STEPS_REWARD,
                       warm_up_steps=WARM_UP_STEPS,
                       alpha_sampling=ALPHA_SAMPLING,
                       beta_sampling=BETA_SAMPLING,
                       tau=TAU)

2023-10-14 09:52:50.950648: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 09:52:51.751934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-10-14 09:52:51.751989: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-10-14 09:52:51.751993: W 

In [7]:
trainer.train(15000)

  0%|          | 0/15000 [00:00<?, ?it/s]

> /home/ubuntu/datalab/ChessFormerRL/reinforcement/dqn/dqn_v2.py(130)update_action_data_buffer()
    128             import pdb; pdb.set_trace()
    129 
--> 130         for element in self.previous_actions_data:
    131             element['reward'] += current_reward
    132 

     89     @t.no_grad()
     90     def update_action_data_buffer(self,
     91                                   model_inputs,
     92                                   current_action_index,
     93                                   estimated_action_value,
     94                                   current_reward):
     95         """
     96         updates previous state target with the maximum q_hat value
     97 
     98         Args:
     99             model_inputs (dict[str, torch.tensor]): model inputs
    100             current_action_index (int): index of chosen action
    101             estimated_action_value (float): Q-value associated to the current action
    102             current_reward (floa

  0%|          | 0/15000 [00:39<?, ?it/s]


In [9]:
t.save(model_1.state_dict(), 'runs/experiment_15/model.pt')